In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from feature_bagging import FeatureBaggingWithHyperparamTuning
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import datetime
from sklearn.preprocessing import MinMaxScaler

#PASOS PENDIENTES
MODIFICAR STOCK A MES ACTUAL
ESTANDARIZAR TN ANTES
ESTANDARIZAR TODAS LAS DE CANTIDAD??

In [10]:
df_tb_productos = pd.read_csv('../data/tb_productos.txt', sep='\t')
df_sell_in = pd.read_csv('../data/sell-in.txt', sep='\t')
df_predecir = pd.read_csv('../data/productos_a_predecir.txt', sep='\t')
df_tb_stocks = pd.read_csv('../data/tb_stocks.txt', sep='\t')

In [11]:
# Función para escalar y devolver una serie
def minmax_scale_group(group):
    scaler = MinMaxScaler()
    scaled_values = scaler.fit_transform(group.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Almacenar el escalador para este grupo
    return pd.Series(scaled_values, index=group.index)

# Función para desescalar y devolver una serie
def inverse_minmax_scale_group(group):
    scaler = scalers[group.name]
    inversed_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    return pd.Series(inversed_values, index=group.index)

# Definir la métrica personalizada
def multinacional_metric(y_true, y_pred):
    return abs(sum(y_true - y_pred)) / sum(y_true)

In [12]:
#### MARCOS
df_tb_stocks['periodo'] = pd.to_datetime(df_tb_stocks['periodo'], format='%Y%m')

df_sell_in = df_sell_in[df_sell_in['periodo']<201808]
df_sell_in['tn_2'] = df_sell_in['tn'].shift(-2) #OBTENGO TN 2 MESES ADELANTE
df_sell_in['tn_2'] = np.where(df_sell_in['periodo'].isin([201805, 201804]), np.nan, df_sell_in['tn_2'])
df_sell_in = df_sell_in[df_sell_in['periodo']<201806]

In [14]:
df_tb_stocks['product_id'] = df_tb_stocks['product_id'].astype(int)
df_tb_stocks['stock_final'] = df_tb_stocks['stock_final'].astype(float)
df_tb_productos['product_id'] = df_tb_productos['product_id'].astype(int)
df_tb_productos['sku_size'] = df_tb_productos['sku_size'].astype(int)


df_sell_in['periodo'] = pd.to_datetime(df_sell_in['periodo'], format='%Y%m')
df_sell_in['product_id'] = df_sell_in['product_id'].astype(int)
df_sell_in['customer_id'] = df_sell_in['customer_id'].astype(int)
df_sell_in['cust_request_qty'] = df_sell_in['cust_request_qty'].astype(int)
df_sell_in['cust_request_tn'] = df_sell_in['cust_request_tn'].astype(float)
df_sell_in['tn'] = df_sell_in['tn'].astype(float)
df_sell_in['plan_precios_cuidados'] = df_sell_in['plan_precios_cuidados'].astype(bool)

# Join tb_productos to sell_in on product_id
df_sell_in_merged = pd.merge(df_sell_in, df_tb_productos, on='product_id', how='left')
# Join tb_stocks to sell_in_merged on both product_id and periodo
df_final = pd.merge(df_sell_in_merged, df_tb_stocks, on=['product_id', 'periodo'], how='left')

df_final['fecha'] = pd.to_datetime(df_final['periodo'], format='%Y%m')
df_final["trimestre"] = df_final.periodo.dt.quarter

#all_periods = pd.DataFrame()
#all_periods['fecha'] = pd.date_range(start='2017-01-01', periods=36, freq='MS')
#df_final = pd.merge(all_periods, df_final, on=['fecha'], how='left')

df_final.set_index('fecha', inplace=True)
df_final.index = df_final.index.to_period('M')

In [15]:
df_final['diff_tn_tn2'] =  df_final['tn_2'] - df_final['tn'] #NUEVA CLASE

In [16]:
#Aplico lags

In [17]:
n_lags = 36

In [18]:
# Create lag variables for 'cust_request_qty', 'cust_request_tn', and 'tn' from lag -1 to -12
for lag in range(1, n_lags + 1):
    #df_final[f'cust_request_qty_lag_{lag}'] = df_final['cust_request_qty'].shift(lag)
    df_final[f'cust_request_tn_lag_{lag}'] = df_final['cust_request_tn'].shift(lag)
    #df_final[f'stock_vendido_lag_{lag}'] = df_final['stock_vendido'].shift(lag)
    df_final[f'stock_final_lag_{lag}'] = df_final['stock_final'].shift(lag)
    df_final[f'tn_lag_{lag}'] = df_final['tn'].shift(lag)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\462342446.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'cust_request_tn_lag_{lag}'] = df_final['cust_request_tn'].shift(lag)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\462342446.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'stock_final_lag_{lag}'] = df_final['stock_final'].shift(lag)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\462342446.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fra

#INNECESARIO!!!!
#Asignacion de nombres de columna
columns_to_sum = ['tn']

for lag in range(1, n_lags + 1):
    columns_to_sum.append(f'cust_request_tn_lag_{lag}')
    columns_to_sum.append(f'stock_final_lag_{lag}')
    columns_to_sum.append(f'tn_lag_{lag}')
						
# Agrupar y sumar las columnas deseadas
#df_final = df_final.groupby(['fecha', 'product_id'])[columns_to_sum].sum().reset_index()


In [ ]:
#df_final.set_index('fecha', inplace=True)

In [19]:
#DELTA DE LAGS 2 ANTERIORES

for lag in range(1, n_lags):
    df_final[f'delta_cust_request_tn_{lag}_{lag+2}'] = (
        df_final[f'cust_request_tn_lag_{lag+2}'] - df_final[f'cust_request_tn_lag_{lag}']
    )
    df_final[f'delta_stock_final_{lag}_{lag+2}'] = (
        df_final[f'stock_final_lag_{lag+2}'] - df_final[f'stock_final_lag_{lag}']
    )
    df_final[f'delta_tn_{lag}_{lag+2}'] = (
        df_final[f'tn_lag_{lag+2}'] - df_final[f'tn_lag_{lag}']
    )

C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\2218199939.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_cust_request_tn_{lag}_{lag+2}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\2218199939.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_stock_final_{lag}_{lag+2}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\2218199939.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

KeyError: 'cust_request_tn_lag_37'

In [20]:
#DELTA DE LAGS 1 ANTERIOR

for lag in range(1, n_lags):
    df_final[f'delta_cust_request_tn_{lag}_{lag+1}'] = (
        df_final[f'cust_request_tn_lag_{lag+1}'] - df_final[f'cust_request_tn_lag_{lag}']
    )
    df_final[f'delta_stock_final_{lag}_{lag+1}'] = (
        df_final[f'stock_final_lag_{lag+1}'] - df_final[f'stock_final_lag_{lag}']
    )
    df_final[f'delta_tn_{lag}_{lag+1}'] = (
        df_final[f'tn_lag_{lag+1}'] - df_final[f'tn_lag_{lag}']
    )

C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\1785296116.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_cust_request_tn_{lag}_{lag+1}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\1785296116.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'delta_stock_final_{lag}_{lag+1}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\1785296116.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

In [21]:
df_final.head(3)

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,tn_2,cat1,cat2,...,delta_tn_32_33,delta_cust_request_tn_33_34,delta_stock_final_33_34,delta_tn_33_34,delta_cust_request_tn_34_35,delta_stock_final_34_35,delta_tn_34_35,delta_cust_request_tn_35_36,delta_stock_final_35_36,delta_tn_35_36
fecha,,,,,,,,,,,,,,,,,,,,,
2017-01,2017-01-01,10234,20524,False,2,0.05300,0.05300,0.03028,HC,VAJILLA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01,2017-01-01,10032,20524,False,1,0.13628,0.13628,0.02271,HC,VAJILLA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01,2017-01-01,10217,20524,False,1,0.03028,0.03028,1.54452,HC,VAJILLA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#Mes actual / (lag 2 + lag 3) 

# Calcular el ratio del valor actual con respecto a la suma de los dos lags anteriores
for lag in range(3, n_lags + 1):
    df_final[f'ratio_cust_request_tn_{lag}'] = (
        df_final[f'cust_request_tn_lag_{lag}'] / (
            df_final[f'cust_request_tn_lag_{lag-1}'] + df_final[f'cust_request_tn_lag_{lag-2}']
        )
    )
    df_final[f'ratio_stock_final_{lag}'] = (
        df_final[f'stock_final_lag_{lag}'] / (
            df_final[f'stock_final_lag_{lag-1}'] + df_final[f'stock_final_lag_{lag-2}']
        )
    )
    df_final[f'ratio_tn_{lag}'] = (
        df_final[f'tn_lag_{lag}'] / (
            df_final[f'tn_lag_{lag-1}'] + df_final[f'tn_lag_{lag-2}']
        )
    )

C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\2974401302.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'ratio_cust_request_tn_{lag}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\2974401302.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'ratio_stock_final_{lag}'] = (
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\2974401302.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

In [23]:
df_final.reset_index(inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\3586226101.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final.reset_index(inplace=True)


In [24]:
#Calcular ventas por trimestre
df_final['tn_trimestre'] = df_final.groupby(['trimestre', 'product_id'])['tn'].transform('sum')
#Calcular ventas por trimestre por cliente
df_final['tn_trimestre_customer'] = df_final.groupby(['trimestre','customer_id', 'product_id'])['tn'].transform('sum')

C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\3111183930.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['tn_trimestre'] = df_final.groupby(['trimestre', 'product_id'])['tn'].transform('sum')
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\3111183930.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['tn_trimestre_customer'] = df_final.groupby(['trimestre','customer_id', 'product_id'])['tn'].transform('sum')


In [25]:
#MEDIAS MOVILES
rolling_windows = [3, 6, 9, 12, 24, 36]

# Agrupamos por 'product_id' y calculamos las medias móviles para 'tn'
for window in rolling_windows:
    df_final[f'rolling_mean_tn_{window}'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())

C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\3749147370.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'rolling_mean_tn_{window}'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\3749147370.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'rolling_mean_tn_{window}'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())
C:\Users\Admin\AppData\Local\Temp\

In [26]:
#MEDIAS MOVILES POR CLIENTE
rolling_windows = [3, 6, 9, 12, 24, 36]

# Agrupamos por 'product_id' y calculamos las medias móviles para 'tn'
for window in rolling_windows:
    df_final[f'rolling_mean_tn_customer_{window}'] = df_final.groupby(['product_id','customer_id'])['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())

C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\3819369804.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'rolling_mean_tn_customer_{window}'] = df_final.groupby(['product_id','customer_id'])['tn'].transform(lambda x: x.rolling(window, min_periods=1).mean())
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\3819369804.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'rolling_mean_tn_customer_{window}'] = df_final.groupby(['product_id','customer_id'])['tn'].transform(lambda x: x.rolling(window, min_per

In [ ]:
#FE CARLA

In [27]:
#Variables Dummies si es el max o el min de cierta cantidad de meses
months = [3, 6, 9, 12]

# Agrupamos por 'product_id' y calculamos las medias móviles para 'tn'
for i in months:
    df_final[f'max_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).max())
    df_final[f'min_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).min())

    # Crear las dummies
    df_final[f'dummy_max_{i}m'] = np.where(df_final['tn'] == df_final[f'max_{i}m'], 1, 0)
    df_final[f'dummy_min_{i}m'] = np.where(df_final['tn'] == df_final[f'min_{i}m'], 1, 0)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\525506276.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'max_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).max())
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\525506276.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f'min_{i}m'] = df_final.groupby('product_id')['tn'].transform(lambda x: x.rolling(i, min_periods=1).min())
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\525506276.py:10: PerformanceWar

In [28]:
df_final['tn_product_id'] = df_final.groupby(['periodo', 'product_id'])['tn'].transform('sum')

C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\4104376824.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['tn_product_id'] = df_final.groupby(['periodo', 'product_id'])['tn'].transform('sum')


#Delta entre lags 

FE para captar tendencia y estacionalidad
Calcular ventas por trimestre
Calcular q de días que faltan para finalizar trimestre
Ratio de ventas vs ventas año anterior
Aplicar un coeficiente a cada mes el peso que tiene respecto a los 12 meses
Después de estandarizar con algún campo hay marcar los ceros
Crear media ponderada para que los valores más recientes tengan mayor peso
Medición la velocidad de caída
Dummie si es el max o el min de cierta cantidad de meses
MEDIAS MOVILES
EXTERNAS


In [29]:
df_final['month'] = df_final['periodo'].dt.month
df_final['year'] = df_final['periodo'].dt.year

df_final['plan_precios_cuidados'] =df_final['plan_precios_cuidados'].astype(bool)
#df_final['dias_fin_trimestre'] = df_final['dias_fin_trimestre'].dt.days.astype(int)
df_final["trimestre"] = df_final.periodo.dt.quarter

#df_final = df_final.drop(columns=['periodo'])
df_final = df_final.drop(columns=['fecha'])

C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\1234819182.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['month'] = df_final['periodo'].dt.month
C:\Users\Admin\AppData\Local\Temp\ipykernel_9128\1234819182.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['year'] = df_final['periodo'].dt.year


CREACION ARCHIVO PARQUET

In [34]:
df_parquet = df_final
df_parquet["cat1"] = df_parquet["cat1"].astype("str")
df_parquet["cat2"] = df_parquet["cat2"].astype("str")
df_parquet["cat3"] = df_parquet["cat3"].astype("str")
df_parquet["brand"] = df_parquet["brand"].astype("str")

df_parquet.to_parquet('./FE_dataset2018.parquet', engine='pyarrow')  

LECTURA DE PARQUET

In [35]:
df_final = pd.read_parquet('FE_dataset2018.parquet')  

In [36]:
df_final["cat1"] = df_final["cat1"].astype("category")
df_final["cat2"] = df_final["cat2"].astype("category")
df_final["cat3"] = df_final["cat3"].astype("category")
df_final["brand"] = df_final["brand"].astype("category")

scalers = {}

df_final['tn'] = df_final.groupby('product_id')['tn'].transform(minmax_scale_group) #escalado
df_final['tn_2'] = df_final.groupby('product_id')['tn_2'].transform(minmax_scale_group) #escalado

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\_array_api.py:472: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\_array_api.py:489: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\_array_api.py:472: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\_array_api.py:489: RuntimeWarning: All-NaN slice encountered


In [37]:
df_final["descripcion"] = df_final["descripcion"].astype("category")

KeyError: 'descripcion'

In [38]:
#data = df_final.loc['2018-01-01':'2020-01-01']

data = df_final #[(df_final['periodo']>=datetime.datetime(year=2019,month=1,day=1)) & (df_final['periodo']<=datetime.datetime(year=2019,month=12,day=1)) ]
data = data.drop(columns=['periodo'])

#data = data[data['product_id'] == 20001] 
#data = data.copy()
#data.fillna(0, inplace=True)
X = data.drop(columns=['diff_tn_tn2'])
#X = X.copy()
y = data['diff_tn_tn2']
#y= y.copy()

#true =df_final.loc['2019-11-01':'2019-12-01']
#true = true[true['product_id'] == 20001] 

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Crear dataset de LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Definir parámetros del modelo
params = {
    'objective': 'regression',
    'metric': 'mae',
}

# Entrenar el modelo
model = lgb.train(params, 
                  train_data, 
                  valid_sets=[test_data],
                  num_boost_round=100,
                  callbacks=[lgb.early_stopping(stopping_rounds=10)])

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.903599 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 78773
[LightGBM] [Info] Number of data points in the train set: 1187018, number of used features: 325
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[8]	valid_0's l1: 0.273587


In [92]:
data_predecir = df_final[df_final['periodo']==datetime.datetime(year=2018,month=5,day=1)] 

data_predecir = data_predecir[data_predecir['product_id'].isin(df_predecir['product_id'])]

data_predecir = data_predecir.drop(columns=['periodo'])

X_predecir = data_predecir.drop(columns=['diff_tn_tn2'])

data_predecir['diff_tn_tn2'] = model.predict(X_predecir)

In [93]:
# Aplicar la desescalado a cada grupo de productos
data_predecir['tn'] = data_predecir.groupby('product_id')['tn'].transform(inverse_minmax_scale_group)

# Aplicar la desescalado a cada grupo de productos
#data_predecir['tn_2'] = data_predecir.groupby('product_id')['tn_2'].transform(inverse_minmax_scale_group)

data_predecir['tn_2'] = data_predecir['tn'] + data_predecir['diff_tn_tn2']

data_predecir = data_predecir.groupby(['product_id'])['tn_2'].agg('sum')
data_predecir = data_predecir.reset_index()
data_predecir = data_predecir.rename(columns={'tn_2': 'tn'}) #PREDICCION TN DE 2 MESES ADELANTE
 

In [96]:
data_predecir2 = data_predecir.rename(columns={'tn': 'pred'})

In [97]:
data_predecir2

,product_id,pred
0,20001,676.609643
1,20002,575.128478
2,20003,393.217578
3,20004,332.634477
4,20005,325.715139
...,...,...
594,21207,-0.349771
595,21212,-0.553050
596,21218,0.831431
597,21222,0.009723


In [107]:
#SI QUIERO COMPARAR CONTRA DICIEMBRE



df_sell_in_diciembre = pd.read_csv('data/sell-in.txt', sep='\t')
df_sell_in_diciembre = df_sell_in_diciembre[df_sell_in_diciembre['periodo'] == 201807]

valores_reales_diciembre = df_sell_in_diciembre
valores_reales_diciembre = valores_reales_diciembre.groupby(['product_id'])['tn'].agg('sum')

# Combinar predicciones y valores reales
resultados = data_predecir2.merge(valores_reales_diciembre, on='product_id')

productos = df_tb_productos.loc[:, ['product_id','cat1']]

resultados = resultados.merge(productos, on='product_id')

# Calcular la métrica
resultados = resultados[resultados['cat1']=='HC']
y_true = resultados['tn']
y_pred = resultados['pred']
metric_result = multinacional_metric(y_true, y_pred)
metric_result




0.3344027311148189

In [104]:
resultados

,product_id,pred,tn,cat1
0,20001,676.609643,1470.41009,HC
1,20002,575.128478,977.40239,HC
2,20003,393.217578,656.22700,FOODS
3,20004,332.634477,653.42310,FOODS
4,20005,325.715139,502.34077,FOODS
...,...,...,...,...
594,21207,-0.349771,0.21173,PC
595,21212,-0.553050,0.16847,HC
596,21218,0.831431,0.16725,REF
597,21222,0.009723,0.09829,REF


In [ ]:
# Especificar las columnas a mantener
columnas_a_mantener = ['product_id','tn']

# Seleccionar solo las columnas especificadas
df_predecir2 = df_predecir2.loc[:, columnas_a_mantener]

In [ ]:
df_predictions = pd.DataFrame(data_predecir)
df_predictions['tn'] = df_predictions['tn'].clip(lower=0)
df_predictions = df_predictions.fillna(0)
df_predictions.to_csv('ESCALADO_TN2.csv', index=False, header=True)

In [ ]:
df_predictions

In [ ]:
#LIGHTGBM MARCOS

In [ ]:
predictions = []
# Define parameter bounds for Bayesian optimization
param_bounds = {
    'num_leaves': (20, 40),
    'learning_rate': (0.01, 0.1),
    'n_estimators': (50, 200),
    'min_child_samples': (5, 30),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'max_depth': (5, 15)
}

products = data['product_id'].unique()
    # Ajustar y predecir para cada producto
for producto in products:
    print(f'Fitting and predicting for product_id: {producto}')
    # Filtrar los datos del producto
    df_producto = data[data['product_id'] == producto]
    df_producto = df_producto['tn'].copy()
    df_producto = df_producto.reset_index()

    # Create the feature bagging model
    feature_bagging_model = FeatureBaggingWithHyperparamTuning(
        X, y, n_models=10, feature_fraction=1, sample_fraction=1, param_bounds=param_bounds, random_state=30000841,optimization_target=multinacional_metric)

    # Fit the model with a single seed
    feature_bagging_model.fit()
    single_seed_predictions = feature_bagging_model.predict(X)

    # Define multiple seeds
    #seeds = [10000019, 20000379, 30000841, 40001387, 50001863]
     # Fit the model with multiple seeds and get combined predictions
    #combined_predictions = feature_bagging_model.fit_multiple_seeds(seeds)
   
    # Realizar el pronóstico para 2 meses adelante
    forecast = feature_bagging_model.forecast(X, n_periods=2)
    
    # Obtener la predicción del segundo mes
    second_month_prediction = forecast[1]  # .iloc[1] obtiene el segundo valor predicho

    # Almacenar el producto_id y la predicción en la lista
    predictions.append({'product_id': producto, 'tn': second_month_prediction})
    print({'product_id': producto, 'tn': second_month_prediction})
    print(true[true['product_id'] == producto]['tn'])

,cat1,cat2,cat3,brand,sku_size,product_id,descripcion
0,FOODS,ADEREZOS,Aji Picante,NATURA,240,20609,Salsa Aji Picante
1,FOODS,ADEREZOS,Barbacoa,NATURA,250,20266,Salsa Barbacoa
2,FOODS,ADEREZOS,Barbacoa,NATURA,400,20325,Salsa Barbacoa
3,FOODS,ADEREZOS,Barbacoa,NATURA,500,20503,Salsa Barbacoa
4,FOODS,ADEREZOS,Chimichurri,NATURA,350,20797,Chimichurri
...,...,...,...,...,...,...,...
1246,REF,TE,Frutas,TWININGS,20,21271,Frutas
1247,REF,TE,Hierbas,TWININGS,20,21202,Manzanilla
1248,REF,TE,Hierbas,TWININGS,20,21218,Menta
1249,REF,TE,Verde,TWININGS,20,21192,Verde
